# Diploma Thesis Design 1
## Named Entity Recognition


In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:60% !important; }</style>"))
%config IPCompleter.greedy=True

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%pylab inline
import tensorflow as tf


Populating the interactive namespace from numpy and matplotlib


In [3]:
import re
from pathlib import Path

## Importing the data from the filesystem

(all the data are from the *nlpprogress.com* website)

### Explaining the data structure:
- First column contains one word in each row
- Second column contains the part-of-speech tag of the word
- Third column contains the chunk tag of the word
- Fourth column contains the named entity tag of the word

- There are 4 types of named entities in the data
    - Person (PER)
    - Location (LOC)
    - Organization (ORG)
    - Miscellaneous/Other (MISC)


In [4]:
train_data = pd.read_csv(r'E:\Egyetem\Diplomaterv\data\conllpp_train.txt', sep=" ", names=['word', 'part-of-speech', 'chunk', 'named entity'])
dev = pd.read_csv(r'E:\Egyetem\Diplomaterv\data\conllpp_dev.txt', sep=" ", names=['word', 'part-of-speech', 'chunk', 'named entity'])
test = pd.read_csv(r'E:\Egyetem\Diplomaterv\data\conllpp_test.txt', sep=" ", names=['word', 'part-of-speech', 'chunk', 'named entity'])

### Importing the data in another way

In [5]:
def read_and_sentence(file_path):
    file_path = Path(file_path)
    
    raw_text = file_path.read_text().strip()
    raw_sentences = re.split(r'\n\t?\n', raw_text)
    
    sentence_tokens = []
    sentence_tags = []
    
    for sents in raw_sentences:
        tokens = []
        tags = []
        
        for line in sents.split('\n'):
            token = line.split()[0]
            tag = line.split()[3]
            
            tokens.append(token)
            tags.append(tag)
            
        sentence_tokens.append(tokens)
        sentence_tags.append(tags)
    
    return sentence_tokens, sentence_tags

In [6]:
texts, tags = read_and_sentence('E:\Egyetem\Diplomaterv\data\conllpp_train.txt')

In [7]:
tags[:5]

[['O'],
 ['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O'],
 ['B-PER', 'I-PER'],
 ['B-LOC', 'O'],
 ['O',
  'B-ORG',
  'I-ORG',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-MISC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-MISC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O']]

In [8]:
texts[:5]

[['-DOCSTART-'],
 ['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'],
 ['Peter', 'Blackburn'],
 ['BRUSSELS', '1996-08-22'],
 ['The',
  'European',
  'Commission',
  'said',
  'on',
  'Thursday',
  'it',
  'disagreed',
  'with',
  'German',
  'advice',
  'to',
  'consumers',
  'to',
  'shun',
  'British',
  'lamb',
  'until',
  'scientists',
  'determine',
  'whether',
  'mad',
  'cow',
  'disease',
  'can',
  'be',
  'transmitted',
  'to',
  'sheep',
  '.']]

In [9]:
train_data

,word,part-of-speech,chunk,named entity
0,-DOCSTART-,-X-,-X-,O
1,EU,NNP,B-NP,B-ORG
2,rejects,VBZ,B-VP,O
3,German,JJ,B-NP,B-MISC
4,call,NN,I-NP,O
...,...,...,...,...
204562,three,CD,I-NP,O
204563,Swansea,NN,B-NP,B-ORG
204564,1,CD,I-NP,O
204565,Lincoln,NNP,I-NP,B-ORG


In [10]:
len(train_data)

204567

In [11]:
train_data.tail(5)

,word,part-of-speech,chunk,named entity
204562,three,CD,I-NP,O
204563,Swansea,NN,B-NP,B-ORG
204564,1,CD,I-NP,O
204565,Lincoln,NNP,I-NP,B-ORG
204566,2,CD,I-NP,O


In [12]:
postags = train_data.groupby("part-of-speech")
## words = train.groupby("word")

In [13]:
ner_unique = train_data.groupby("named entity")
ner_unique.first()

,word,part-of-speech,chunk
named entity,,,
B-LOC,BRUSSELS,NNP,B-NP
B-MISC,German,JJ,B-NP
B-ORG,EU,NNP,B-NP
B-PER,Peter,NNP,B-NP
I-LOC,Strait,NNP,I-NP
I-MISC,Spongiform,NNP,I-NP
I-ORG,Commission,NNP,I-NP
I-PER,Blackburn,NNP,I-NP
O,-DOCSTART-,-X-,-X-


In [14]:
vmi = train_data[train_data.word == "-DOCSTART-"]
vmi

,word,part-of-speech,chunk,named entity
0,-DOCSTART-,-X-,-X-,O
470,-DOCSTART-,-X-,-X-,O
660,-DOCSTART-,-X-,-X-,O
900,-DOCSTART-,-X-,-X-,O
977,-DOCSTART-,-X-,-X-,O
...,...,...,...,...
203676,-DOCSTART-,-X-,-X-,O
203867,-DOCSTART-,-X-,-X-,O
204161,-DOCSTART-,-X-,-X-,O
204273,-DOCSTART-,-X-,-X-,O


In [15]:
postags.first()

,word,chunk,named entity
part-of-speech,,,
$,$,I-NP,O
'',',I-NP,O
(,(,O,O
),),O,O
",",",",O,O
-X-,-DOCSTART-,-X-,O
.,.,O,O
:,--,O,O
CC,and,O,O


In [16]:
postags.describe()

word                             chunk                        \
                count unique           top  freq  count unique     top   freq   
part-of-speech                                                                  
$                 427     10             $   362    427      3    B-NP    325   
''                 35      1             '    35     35      2       O     30   
(                2866      2             (  2861   2866      3       O   2788   
)                2866      2             )  2861   2866      2       O   2851   
,                7291      2             ,  7290   7291      7       O   6961   
-X-               946      1    -DOCSTART-   946    946      1     -X-    946   
.                7389      3             .  7374   7389      2       O   7344   
:                2386      5             -  1243   2386      5       O   1878   
CC               3653     19           and  2838   3653      7       O   2642   
CD              19704   3491             1  1420  19704      9    I-NP  13553   
DT              13453     47           the  7243  13453      7    B-NP  13202   
EX                136      2         there    81    136      2    B-NP    135   
FW                166     52             v    47    166      7    I-NP     93   
IN              19064    150            of  3751  19064     14    B-PP  17294   
JJ              11831   2853         first   349  11831     12    I-NP   6689   
JJR               382     76          more   100    382      6    B-NP    212   
JJS               254     41          most    39    254      6    I-NP    166   
LS                 13     10           212     4     13      3   B-LST      8   
MD               1199     20          will   410   1199      3    B-VP   1184   
NN              23898   4345       percent   301  23899     10    I-NP  18866   
NNP             34390   8411          U.S.   377  34392      8    B-NP  17301   
NNPS              684    317  Palestinians    20    684      2    I-NP    516   
NNS              9903   2078        people   175   9903      5    I-NP   7011   
NN|SYM              4      2           TVM     3      4      1    B-NP      4   
O                2178      1                2178   2178      1       O   2178   
PDT                33      7           all    20     33      4    B-NP     26   
POS              1553      3            's  1417   1553      3    B-NP   1481   
PRP              3163     43            he   582   3163      2    B-NP   3136   
PRP$             1520     18           his   538   1520      2    B-NP   1495   
RB               3975    530           not   536   3975     16  B-ADVP   1878   
RBR               163     15       earlier    58    163      8    B-NP     52   
RBS                35      4          most    31     35      4    I-NP     23   
RP                528     20            up   177    528      6   B-PRT    506   
SYM               439     19             /   235    439      4       O    421   
TO               3469     11            to  3382   3469      6    B-PP   1308   
UH                 30     24             O     4     30      3  B-INTJ     27   
VB               4252    880            be   525   4252      6    I-VP   3379   
VBD              8293    819          said  1835   8293      6    B-VP   8131   
VBG              2585    783         being    86   2585      6    B-VP   1597   
VBN              4105    990          been   368   4105      8    I-VP   2778   
VBP              1436    252           are   346   1436      5    B-VP   1418   
VBZ              2426    389            is   691   2426      4    B-VP   2380   
WDT               506      8         which   330    506      1    B-NP    506   
WP                528      8           who   439    528      2    B-NP    526   
WP$                23      1         whose    23     23      2    B-NP     22   
WRB               384      8          when   220    384      5  B-ADVP    378   

               named entity                    

In [17]:
full_train = train_data
train_data = train_data.head(10000)

full_dev = dev
dev = dev.head(10000)

full_test = test
test = test.head(10000)


In [18]:
def label_to_number(data):
    entity = data["named entity"]
    label = 0
    if entity == "O":
        label = 0
    elif entity == "B-LOC":
        label = 1
    elif entity == "I-LOC":
        label = 2
    elif entity == "B-MISC":
        label = 3
    elif entity == "I-MISC":
        label = 4
    elif entity == "B-PER":
        label = 5
    elif entity == "I-PER":
        label = 6
    elif entity == "B-ORG":
        label = 7
    elif entity == "I-ORG":
        label = 8
    
    return label
        

## First model:

In [19]:
import torch
from torch import nn
import torch.optim as optim
from sklearn.model_selection import train_test_split as split
from torch.utils.data import DataLoader

In [20]:
train_word_list = []
train_label_list = []
for index, row in train_data.iterrows():
    train_word_list.append(row.word)
    train_label_list.append(label_to_number(row))

In [21]:
dev_word_list = []
dev_label_list = []
for index, row in dev.iterrows():
    dev_word_list.append(row.word)
    dev_label_list.append(label_to_number(row))

In [22]:
test_word_list = []
test_label_list = []
for index, row in test.iterrows():
    test_word_list.append(row.word)
    test_label_list.append(label_to_number(row))

In [23]:
print(train_label_list[:10])
print(train_word_list[:10])
print(type(train_label_list))

[0, 7, 0, 3, 0, 0, 0, 3, 0, 0]
['-DOCSTART-', 'EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']
<class 'list'>


In [24]:
train_word_list = pd.Series(train_word_list).fillna("").tolist()

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

vectorizer = CountVectorizer()

word_to_ix = vectorizer.fit(train_word_list)

In [26]:
VOCAB_SIZE = len(word_to_ix.vocabulary_)
OUT_DIM = len(ner_unique.first())

BATCH_SIZE = 256
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [29]:
tr_data, val_data = train_word_list, test_word_list

tr_data_vecs = torch.FloatTensor(word_to_ix.transform(tr_data).toarray())
val_data_vecs = torch.FloatTensor(word_to_ix.transform(val_data).toarray())

tr_labels = train_label_list
val_labels = test_label_list

In [46]:
tr_data_loader = [(sample, label) for sample, label in zip(tr_data_vecs, tr_labels)]
val_data_loader = [(sample, label) for sample, label in zip(val_data_vecs, val_labels)]

train_iterator = DataLoader(tr_data_loader,
                            batch_size=BATCH_SIZE,
                            shuffle=True,
                            )

valid_iterator = DataLoader(val_data_loader,
                          batch_size=BATCH_SIZE,
                          shuffle=False,
                          )

In [31]:
print(type(tr_labels))

<class 'list'>


In [47]:
class FNN(nn.Module):
    def __init__(self, num_labels, vocab_size):
        super(FNN, self).__init__()
    
        self.hidden1 = nn.Linear(vocab_size, 250)
        self.act1 = nn.ReLU()
        
        self.hidden2 = nn.Linear(250, 100)
        self.act2 = nn.ReLU()
        
        self.hidden3 = nn.Linear(100, num_labels)
        self.act3 = nn.Sigmoid()
    
    def forward(self, bow_vec):
        bow_vec = self.hidden1(bow_vec)
        bow_vec = self.act1(bow_vec)
        
        bow_vec = self.hidden2(bow_vec)
        bow_vec = self.act2(bow_vec)
        
        bow_vec = self.hidden3(bow_vec)
        bow_vec = self.act3(bow_vec)
        return F.log_softmax(bow_vec, dim=1)

In [48]:
model = FNN(OUT_DIM, VOCAB_SIZE)

In [49]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.NLLLoss()

model = model.to(device)
criterion = criterion.to(device)

In [50]:
def class_accuracy(preds, y):
    
    rounded_preds = preds.argmax(1)
    correct = (rounded_preds == y).float()
    
    acc = correct.sum() / len(correct)
    return acc

In [51]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for texts, labels in iterator:
        texts = texts.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
                  
        predictions = model(texts)

        loss = criterion(predictions, labels)
        acc = class_accuracy(predictions, labels)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [52]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():
        
        for texts, labels in iterator:
            
            texts = texts.to(device)
            labels = labels.to(device)
            
            predictions = model(texts)
            loss = criterion(predictions, labels)
            
            acc = class_accuracy(predictions, labels)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [53]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [54]:
import torch.nn.functional as F

N_EPOCHS = 15

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 0s
	Train Loss: 2.075 | Train Acc: 76.42%
	 Val. Loss: 1.835 |  Val. Acc: 75.21%
Epoch: 02 | Epoch Time: 0m 0s
	Train Loss: 1.651 | Train Acc: 76.90%
	 Val. Loss: 1.623 |  Val. Acc: 75.21%
Epoch: 03 | Epoch Time: 0m 0s
	Train Loss: 1.600 | Train Acc: 76.90%
	 Val. Loss: 1.619 |  Val. Acc: 75.21%
Epoch: 04 | Epoch Time: 0m 0s
	Train Loss: 1.593 | Train Acc: 76.90%
	 Val. Loss: 1.615 |  Val. Acc: 75.21%
Epoch: 05 | Epoch Time: 0m 0s
	Train Loss: 1.584 | Train Acc: 77.37%
	 Val. Loss: 1.607 |  Val. Acc: 77.05%
Epoch: 06 | Epoch Time: 0m 0s
	Train Loss: 1.562 | Train Acc: 80.77%
	 Val. Loss: 1.598 |  Val. Acc: 77.64%
Epoch: 07 | Epoch Time: 0m 0s
	Train Loss: 1.530 | Train Acc: 83.64%
	 Val. Loss: 1.591 |  Val. Acc: 79.09%
Epoch: 08 | Epoch Time: 0m 0s
	Train Loss: 1.511 | Train Acc: 86.52%
	 Val. Loss: 1.584 |  Val. Acc: 79.16%
Epoch: 09 | Epoch Time: 0m 0s
	Train Loss: 1.484 | Train Acc: 88.26%
	 Val. Loss: 1.578 |  Val. Acc: 79.23%
Epoch: 10 | Epoch Time: 0m 0